<a href="https://colab.research.google.com/github/scienclick/src/blob/master/Bitumen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:

# <editor-fold desc="variables definition">
components = ['Brine', 'Bitumen', 'Gas']
tc_mob = 80.00
a = 30
freq_dflt = 150

# 2nd block -- variables
rs = 0.01      #solution gas ratio scf/STB
g_g = 0.7      # gas gravity


rho_quartz = 2.65 #gr/cc
k_quartz = 36. # GPa
mu_quartz = 44. # GPa
k_shale_global = 8.97 #GPa
mu_shale_global = 1.63 #GPa
rho_matrix = 2.65 #gr/cc

###################
stbl_param = 0.75 # orig: 0.566
stbl_param2 = 1.15 # orig: 1.131
###################

javan_coeff = {'B': [0.01859, 0.1922, 0.56221],
               'C': [0, 0.00835, 0.07514],
               'mu-max': 0.826,
               'K-par': [1.08, 0.92],
               'mu-par': [1.07, 0.57],
               'A': 0.104,
               'mua-m': 0.27,
               'So-n': 0.14,
               'Phi-p': -1.9,
               'Mudry-q': -1.76,
               'A2': 0.088,
               'mua-m2': 0.669,
               'So-n2': 0.28,
               'Phi-p2': -4.466,
               'Musat-q2': -3.063}


In [0]:
'''
input:api
output: rho_0
'''
api=10
rho_0=141.5 /(api + 131.5)

In [24]:

'''
inputs: p_pore, api, tc 
outputs: bitumen density
  '''
p_pore=2.5  #psi bar or kb, Gp
api=10
tc=30

rho_p = rho_0 + (0.00277 * p_pore - 1.71e-7 * np.power(p_pore, 3)) * np.power((rho_0 - 1.15), 2) + 0.000349 * p_pore

bitumen_density=rho_p / (0.972 + 0.000381 * np.power((tc + 17.78), 1.175))

print(bitumen_density)


0.9932679730761929


In [25]:

'''
viscosity(api, tc):

'''
api=10
tc=30

temp_faren=tc*1.8+32
viscosity=np.power(10, (-1.23667 * api + 29.6079)) * np.power(temp_faren, (0.465549 * api - 11.09838))
print(viscosity)

59901.71733764903


In [17]:
'''
ln_relaxation_time(freq):
'''
freq=60
omega=(2 * np.pi * freq)
ln_relaxation_time=np.log((viscosity * 0.000000000001 / 0.826 ))

print(ln_relaxation_time)

-16.439400156697214


In [15]:
'''
shear_modulus
'''
B=0.01859*temp_faren+0.1922*np.log10(viscosity)+ 0.56221
C=0.00835*np.log10(viscosity)+0.07514

shear_modulus=0.826 * np.exp(-B* np.exp(-C * ln_relaxation_time))

print(shear_modulus)

1.137109723959188e-09


In [16]:
    """
    Fluid bulk modulus
    :param p_pore:
    :param api:
    :param freq:
    :param tc:
    :return:
    """
k_bitumen_batzle=12 #Gpa
  
javanbakhti_k= 1.08 * np.power(k_bitumen_batzle, 0.92) +1.07 * np.power(shear_modulus, 0.57)

print(shear_modulus)

1.137109723959188e-09


In [29]:
'''
inputs: mu_dry, 
        mu_a, 
        phi, 
        so
output: javanbakhti_mu_sat
'''
mu_a=shear_modulus
stbl_param = 0.75 # orig: 0.566
phi=.25
mu_dry=18

r_max = stbl_param

phi = phi * np.ones_like(r_max)
so=.6

#for phi>0.3 what about less than that?

rhs = (0.104 *
           np.power(mu_a, 0.27) *
           np.power(so, 0.14) *
           np.power(phi, -1.9) *
           np.power(mu_dry, -1.76))
  
javanbakhti_mu_sat=np.reciprocal((1 / mu_dry) - rhs)
print(javanbakhti_mu_sat)


18.010386544082785


In [31]:
#ciz_shapiro(mod_dry, mod_fl, phi, mod0):
    """this is basically general form of the Gassmann's equation. Can be used for calculating either sat. bulk or
    sat. shear modulus of the rock.
    mod: elastic modulus
    mod0: matrix modulus
    """

mu_sat=javanbakhti_mu_sat
mod_fl=shear_modulus
mod_dry=12
mod0=30
rhs=np.power(1/mod_dry - 1/mod0, 2) / (phi * (1/(mod_fl - 1/(mod0) + (1/(mod_dry)))) - 1/(mod0))
rhs = np.power(1/(mod_dry) - 1/(mod0), 2) / (phi * (1/(mod_fl) - 1/(mod0)) + (1/(mod_dry) - 1/(mod0)))
ciz_shapiro_mu_sat=1/((1 / mu_sat) + rhs)
print(ciz_shapiro_mu_sat)

18.010386540394297
